In [1]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project='hello-phase3')

투자지표
- 수익성: '영업이익률', '영업이익＜당기＞'
- 성장성: '매출액증가율', '매출액＜전기＞', '매출액＜당기＞'
- 안정성: '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용'

In [2]:
sql_y = """
select stock_code, corp_name, year, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_year`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

sql_q = """
select stock_code, corp_name, year, quarter, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_quarter`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

df_loaded_y = client.query_and_wait(sql_y).to_dataframe()
df_loaded_q = client.query_and_wait(sql_q).to_dataframe()

df_y = df_loaded_y.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'account_name'])
df_q = df_loaded_q.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'quarter', 'account_name'])

df_y['account_name'] = df_y['account_name'].str.replace('＜당기＞', '')
df_y['account_name'] = df_y['account_name'].str.replace('＜전기＞', '_전기')

df_q['account_name'] = df_q['account_name'].str.replace('＜당기＞', '')
df_q['account_name'] = df_q['account_name'].str.replace('＜전기＞', '_전기')

### 1. [이익] 최근연도 or 최근분기 OPM 5% 이상

In [3]:
recent_year = 2022
df_1_y = df_y.copy()[(df_y['year'] == recent_year) & (df_y['account_name'] == '영업이익률')]

this_year = recent_year + 1
df_1_q = df_q.copy()[(df_q['year'] == this_year) & (df_q['account_name'] == '영업이익률')]
df_1_q = df_1_q[df_1_q['quarter'] != 4]
df_1_q = df_1_q.groupby('stock_code')['amount'].mean().reset_index()

df_1 = pd.merge(df_1_y, df_1_q, on='stock_code', how='inner')
df_1 = df_1[['stock_code', 'corp_name', 'amount_x', 'amount_y']]
df_1.columns = ['stock_code', 'corp_name', 'OPM_2022', 'OPM_2023(연환산)']
df_1['OPM_5%이상'] = (df_1['OPM_2022'] >= 5) | (df_1['OPM_2023(연환산)'] >= 5)
df_1


,stock_code,corp_name,OPM_2022,OPM_2023(연환산),OPM_5%이상
0,042520,한스바이오메드,1.736,-7.385333,False
1,004310,현대약품,4.899,6.030000,True
2,003610,방림,1.380,-4.777667,False
3,008870,금비,-0.003,3.048667,False
4,093380,풍강,3.611,4.147667,False
...,...,...,...,...,...
2309,030790,비케이탑스,-30.166,-51.677500,False
2310,071950,코아스,-0.720,-2.716333,False
2311,078860,아이오케이,-57.015,-61.010667,False
2312,008500,일정실업,-28.106,-7.067333,False


### 2. 이익률 3개년 플러스 성장
  - 20년 대비 22년 OPM 성장율 0% 이상
  - 3point: 21년↑, 22년↑, 2point: 21년↓, 22년↑, 1point: 21년↑, 22년↓

In [4]:
df_2 = df_y.copy()[(df_y['year'].isin([2022, 2021, 2020])) & (df_y['account_name'] == '영업이익률')]
df_2 = df_2.pivot_table(index=['stock_code', 'corp_name'], columns='year', values='amount', aggfunc='first').reset_index()
df_2['OPM_3개년up'] = (df_2[2022] - df_2[2020]) / df_2[2020]
df_2['OPM_3개년'] = (df_2['OPM_3개년up'] > 0)
df_2.columns.name = ''
df_2.columns = ['stock_code', 'corp_name', 'OPM_2020', 'OPM_2021', 'OPM_2022', 'OPM_3개년up', 'OPM_3개년']
df_2['OPM_2021up'] = (df_2['OPM_2021'] - df_2['OPM_2020']) / df_2['OPM_2020']
df_2['OPM_2022up'] = (df_2['OPM_2022'] - df_2['OPM_2021']) / df_2['OPM_2021']

pt3_condition = (df_2['OPM_2021up'] > 0) & (df_2['OPM_2022up'] > 0)
pt2_condition = (df_2['OPM_2021up'] <= 0) & (df_2['OPM_2022up'] > 0)
pt1_condition = (df_2['OPM_2021up'] > 0) & (df_2['OPM_2022up'] <= 0)
df_2['OPM_3개년추이'] = np.where(pt3_condition, 3, 
                              np.where(pt2_condition, 2, 
                                       np.where(pt1_condition, 1, 0)
                                       )
)
                              
df_2

,stock_code,corp_name,OPM_2020,OPM_2021,OPM_2022,OPM_3개년up,OPM_3개년,OPM_2021up,OPM_2022up,OPM_3개년추이
0,000020,동화약품,8.510,7.677,8.788,0.032667,True,-0.097885,0.144718,2
1,000040,KR모터스,1.035,-2.021,-5.326,-6.145894,False,-2.952657,1.635329,2
2,000050,경방,6.638,14.033,8.195,0.234559,True,1.114040,-0.416019,1
3,000070,삼양홀딩스,6.893,11.318,3.990,-0.421152,False,0.641956,-0.647464,1
4,000080,하이트진로,8.797,7.904,7.630,-0.132659,False,-0.101512,-0.034666,0
...,...,...,...,...,...,...,...,...,...,...
2310,452300,캡스톤파트너스,NaN,53.298,58.126,NaN,False,NaN,0.090585,0
2311,453860,에이에스텍,3.860,5.092,14.042,2.637824,True,0.319171,1.757659,3
2312,454910,두산로보틱스,-68.714,-19.158,-29.427,-0.571747,False,-0.721192,0.536016,2
2313,457550,우진엔텍,3.715,15.849,15.184,3.087214,True,3.266218,-0.041958,1


### 3. 당해년도 누적분기 연환산 매출액 YoY -10% 이상

In [5]:
df_3a = df_y.copy()[(df_y['year'] == 2022) & (df_y['account_name'] == '영업이익률')]
df_3b = df_q.copy()[(df_q['year'] == 2023) & (df_q['account_name'] == '영업이익률')]
df_3a

,stock_code,corp_name,year,account_name,amount
19,042520,한스바이오메드,2022,영업이익률,1.736
135,004310,현대약품,2022,영업이익률,4.899
239,003610,방림,2022,영업이익률,1.380
349,008870,금비,2022,영업이익률,-0.003
459,093380,풍강,2022,영업이익률,3.611
...,...,...,...,...,...
250021,030790,비케이탑스,2022,영업이익률,-30.166
250131,071950,코아스,2022,영업이익률,-0.720
250241,078860,아이오케이,2022,영업이익률,-57.015
250351,008500,일정실업,2022,영업이익률,-28.106


In [6]:
df_3b = df_3b[df_3b['quarter'] != 4]
df_3b = df_3b.pivot_table(index=['stock_code', 'corp_name'], columns='quarter', values='amount')
df_3b = df_3b.dropna(how='any').reset_index()
df_3b[2023] = df_3b[[1,2,3]].mean(axis=1)
df_3b

quarter,stock_code,corp_name,1,2,3,2023
0,000020,동화약품,12.254,6.073,3.081,7.136000
1,000040,KR모터스,-17.337,-15.418,-16.594,-16.449667
2,000050,경방,-3.459,-5.043,9.018,0.172000
3,000070,삼양홀딩스,0.929,4.547,4.381,3.285667
4,000080,하이트진로,6.412,1.858,6.641,4.970333
...,...,...,...,...,...,...
2235,450080,에코프로머티,3.570,2.464,-2.866,1.056000
2236,450140,코오롱모빌리티그룹,1.790,2.197,0.880,1.622333
2237,450520,인스웨이브시스템즈,3.936,0.465,6.343,3.581333
2238,452260,한화갤러리아,4.000,3.110,1.668,2.926000


In [7]:
df_3 = pd.merge(df_3a[['stock_code', 'corp_name', 'amount']], df_3b[['stock_code', 2023]], on='stock_code')
df_3.columns = ['stock_code', 'corp_name', 'OPM_2022', 'OPM_2023(연환산)']
df_3['OPM_2023up(연환산)'] = (df_3['OPM_2023(연환산)'] - df_3['OPM_2022']) / df_3['OPM_2022']
df_3['OPM_당해YoY'] = df_3['OPM_2023up(연환산)'] > 0
df_3

,stock_code,corp_name,OPM_2022,OPM_2023(연환산),OPM_2023up(연환산),OPM_당해YoY
0,042520,한스바이오메드,1.736,-7.385333,-5.254224,False
1,004310,현대약품,4.899,6.030000,0.230863,True
2,003610,방림,1.380,-4.777667,-4.462077,False
3,008870,금비,-0.003,3.048667,-1017.222222,False
4,093380,풍강,3.611,4.147667,0.148620,True
...,...,...,...,...,...,...
2233,217620,디딤이앤에프,-8.710,-8.315667,-0.045274,False
2234,071950,코아스,-0.720,-2.716333,2.772685,True
2235,078860,아이오케이,-57.015,-61.010667,0.070081,True
2236,008500,일정실업,-28.106,-7.067333,-0.748547,False


### 4. 매출 최근분기 QoQ -20% 이상

In [8]:
df_4 = df_3b.copy()[['stock_code', 'corp_name', 2, 3]]
df_4['OPM_최근QoQup'] = (df_4[3] - df_4[2]) / df_4[2]
df_4['OPM_최근QoQ'] = df_4['OPM_최근QoQup'] > -0.2
df_4

quarter,stock_code,corp_name,2,3,OPM_최근QoQup,OPM_최근QoQ
0,000020,동화약품,6.073,3.081,-0.492672,False
1,000040,KR모터스,-15.418,-16.594,0.076274,True
2,000050,경방,-5.043,9.018,-2.788221,False
3,000070,삼양홀딩스,4.547,4.381,-0.036508,True
4,000080,하이트진로,1.858,6.641,2.574273,True
...,...,...,...,...,...,...
2235,450080,에코프로머티,2.464,-2.866,-2.163149,False
2236,450140,코오롱모빌리티그룹,2.197,0.880,-0.599454,False
2237,450520,인스웨이브시스템즈,0.465,6.343,12.640860,True
2238,452260,한화갤러리아,3.110,1.668,-0.463666,False


### 매출기준 최종 정리

In [9]:
['stock_code', 'code_name', 'OPM_2020', 'OPM_2021', 'OPM_2021up', 'OPM_2022', 'OPM_2022up', 'OPM_2023(연환산)', 'OPM_2023up(연환산)', 'OPM_5%이상', 'OPM_3개년', 'OPM_3개년추이', 'OPM_당해YoY', 'OPM_최근QoQ']

['stock_code',
 'code_name',
 'OPM_2020',
 'OPM_2021',
 'OPM_2021up',
 'OPM_2022',
 'OPM_2022up',
 'OPM_2023(연환산)',
 'OPM_2023up(연환산)',
 'OPM_5%이상',
 'OPM_3개년',
 'OPM_3개년추이',
 'OPM_당해YoY',
 'OPM_최근QoQ']

In [10]:

df_opm = pd.merge(df_1[['stock_code', 'corp_name', 'OPM_5%이상']], df_2.drop(columns=['corp_name']), how='left', on='stock_code')
df_opm = pd.merge(df_opm, df_3.drop(columns=['corp_name', 'OPM_2022']), how='left', on='stock_code')
df_opm = pd.merge(df_opm, df_4.drop(columns=['corp_name']), how='left', on='stock_code')
df_opm = df_opm[['stock_code', 'corp_name', 'OPM_2020', 'OPM_2021', 'OPM_2021up', 'OPM_2022', 'OPM_2022up', 'OPM_2023(연환산)', 'OPM_2023up(연환산)', 'OPM_5%이상', 'OPM_3개년', 'OPM_3개년추이', 'OPM_당해YoY', 'OPM_최근QoQ']]

In [11]:
df_opm

,stock_code,corp_name,OPM_2020,OPM_2021,OPM_2021up,OPM_2022,OPM_2022up,OPM_2023(연환산),OPM_2023up(연환산),OPM_5%이상,OPM_3개년,OPM_3개년추이,OPM_당해YoY,OPM_최근QoQ
0,042520,한스바이오메드,-4.254,-33.466,6.866949,1.736,-1.051874,-7.385333,-5.254224,False,False,1,False,False
1,004310,현대약품,2.296,-1.124,-1.489547,4.899,-5.358541,6.030000,0.230863,True,True,0,True,False
2,003610,방림,-1.152,4.852,-5.211806,1.380,-0.715581,-4.777667,-4.462077,False,False,0,False,True
3,008870,금비,3.027,4.330,0.430459,-0.003,-1.000693,3.048667,-1017.222222,False,False,1,False,False
4,093380,풍강,0.349,5.495,14.744986,3.611,-0.342857,4.147667,0.148620,False,True,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2315,030790,비케이탑스,-34.667,-41.687,0.202498,-30.166,-0.276369,NaN,NaN,False,False,1,NaN,NaN
2316,071950,코아스,-2.138,-4.213,0.970533,-0.720,-0.829100,-2.716333,2.772685,False,False,1,True,False
2317,078860,아이오케이,-14.071,-38.991,1.771018,-57.015,0.462261,-61.010667,0.070081,False,True,3,True,False
2318,008500,일정실업,-23.436,-27.851,0.188385,-28.106,0.009156,-7.067333,-0.748547,False,True,3,False,False


In [12]:
df_opm['OPM_3개년'] = df_opm['OPM_3개년'].fillna(False)
df_opm['OPM_당해YoY'] = df_opm['OPM_당해YoY'].fillna(False)
df_opm['OPM_최근QoQ'] = df_opm['OPM_최근QoQ'].fillna(False)
df_opm.query('(`OPM_5%이상` == True) and (OPM_3개년 == True) and (OPM_당해YoY == True) and (OPM_최근QoQ == True) and (OPM_3개년추이 == 3)')

,stock_code,corp_name,OPM_2020,OPM_2021,OPM_2021up,OPM_2022,OPM_2022up,OPM_2023(연환산),OPM_2023up(연환산),OPM_5%이상,OPM_3개년,OPM_3개년추이,OPM_당해YoY,OPM_최근QoQ
51,005380,현대차,2.303,5.679,1.465914,6.890,0.213242,9.620667,0.396323,True,True,3,True,True
52,000270,기아,3.493,7.251,1.075866,8.356,0.152393,12.104667,0.448620,True,True,3,True,True
59,028260,삼성물산,2.836,3.471,0.223907,5.858,0.687698,7.040333,0.201832,True,True,3,True,True
79,017670,SK텔레콤,7.761,8.282,0.067131,9.316,0.124849,11.129333,0.194647,True,True,3,True,True
99,012450,한화에어로스페이스,4.584,5.000,0.090750,5.768,0.153600,7.420000,0.286408,True,True,3,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,069730,DSR제강,2.006,2.072,0.032901,10.076,3.862934,10.823333,0.074170,True,True,3,True,True
1966,039610,화성밸브,3.012,3.649,0.211487,4.329,0.186352,11.226333,1.593286,True,True,3,True,True
1977,052600,한네트,5.748,6.348,0.104384,11.340,0.786389,14.556000,0.283598,True,True,3,True,True
2048,046310,백금T&A,4.592,6.361,0.385235,10.393,0.633863,12.300000,0.183489,True,True,3,True,True
